In [ ]:
"""
This notebook retrieves hydrologic data from snow 
telemetry networks in the western United States.

Authors: Tony Castronova, Irene Garousi-Nejad, Raymond Hess

Last modified: 2024-07-19
"""

In [ ]:
### Installing dependencies ###
!pip install suds

In [ ]:
### Importing libraries, pyhis.py and funcs.py are needed in the working directory for this script ###
import pyhis
import funcs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from suds.client import Client

In [ ]:
serv = pyhis.Services()

In [ ]:
### Specify bounding area for watershed of interest ###
minlat = 35.966999
maxlat = 36.7006926958913
minlon = -118.578979
maxlon = -118.267685

sites = serv.get_sites(minlon, minlat, maxlon, maxlat)

In [ ]:
### Previewing available networks ###
set(sites.servCode)

In [ ]:
### Fetching server URL for data network ###
sites[sites.servCode=='CDEC'].iloc[0].servURL

In [ ]:
### Retrieving available site codes ###
sites[sites.servCode=='CDEC'].SiteCode

In [ ]:
### Specifying a link to the data ###
url = 'https://hydroportal.cuahsi.org/CDEC/cuahsi_1_1.asmx'   # url = sites.iloc[0].servURL
sitecode = 'CDEC:PSC'   #sitecode = sites.iloc[0].SiteCode

In [ ]:
### Indexing SNOTEL station information ###
site_info = serv.get_sites_info([url],[sitecode])
site_info

In [ ]:
### Specifying site variable information (SWE changes for some CDEC stations) ###
var = site_info.iloc[3]
var.siteid

In [ ]:
### Creating a new instance of the soap class ###
client = Client(f'{url}?wsdl')

In [ ]:
vals = client.service.GetValuesObject(var.siteid, var.variableCode, var.startdt, var.enddt)

In [ ]:
dates = [] 
values = []
for dat in vals.timeSeries[0].values[0][0]:
    dates.append(dat['_dateTimeUTC'])
    values.append(float(dat['value']))

In [ ]:
### Creating the DataFrame ###
df = pd.DataFrame({'date':dates, f'{var.variableName}':values})

### Converting UTC to Pandas DateTime ###
df['date'] = pd.to_datetime(df['date'], utc=True)

start_date = '2020-10-01'
end_date = '2023-09-30'
mask = (df['date'] >= start_date) & (df['date'] <= end_date)
sliced_df = df.loc[mask]

sliced_df

In [ ]:
### Data Cleaning ###

#sliced_df.loc[sliced_df['Snow water equivalent'] == -9999.0, 'Snow water equivalent'] = 0
sliced_df = sliced_df.drop(sliced_df[sliced_df['Snow water equivalent'] == -9999.0].index)


print(sliced_df)

In [ ]:
### Change Station Name ###
sliced_PSC = (sliced_df.copy())

In [ ]:
### Saving DataFrame to .csv ###
sliced_UTY.to_csv('CBT-SWE.csv')

In [ ]:
### Function for April 1st SWE x-labels ###
def water_year_labels(dates):
    labels = []
    for date in dates:
        if date.month >= 10:  # Water year starts in October
            labels.append(f'WY-{date.year + 1}')
        else:
            labels.append(f'WY-{date.year}')
    return labels

### April 1st SWE x-labels ###
xticks_dates = pd.to_datetime(['2021-04-01', '2022-04-01', '2023-04-01'])
xticks_labels = ['April 1st, 2021', 'April 1st, 2022', 'April 1st, 2023']

### Plotting ###
plt.figure(figsize=(18, 6))
plt.plot(sliced_UTY['date'], sliced_UTY['Snow water equivalent'] * 2.54, color='k', linestyle='-', alpha=0.8, label='Upper Tyndall Creek (UTY)')
plt.plot(sliced_CSV['date'], sliced_CSV['Snow water equivalent'] * 2.54, color='k', linestyle='--', alpha=0.8, label='Casa Vieja Meadows (CSV)')
plt.plot(sliced_PSC['date'], sliced_PSC['Snow water equivalent'] * 2.54, color='k', linestyle='dotted', alpha=0.8, label='Pascoes (PSC)')

plt.ylabel('SWE (cm)', fontsize=15)
plt.xticks(ticks=xticks_dates, labels=xticks_labels, fontsize=15)
plt.yticks(fontsize=15)

plt.legend(fontsize=13)
#plt.title('Peak snow water equivalent, upper Kern River watershed', fontsize=14)
plt.savefig('SWE-SSN', bbox_inches='tight', dpi=300)
plt.show()